## Support Vector Machines

### Table of Contents

- [Packages and Notebook Properties](#notebook)
- [Theory Discussion](#theory)
- [Dataset Exploration](#dataset)
- [Implementation](#implementation)

### Packages and Notebook Properties

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import sys
import os

In [5]:
warnings.filterwarnings('ignore')

### Theory Discussion

### Dataset Exploration

In [ ]:
df = 